# AI_TRANSCRIBEを活用した高度なコールセンター分析

このノートブックでは、SnowflakeのAI_TRANSCRIBE関数と他のCortex AI関数を組み合わせて、コールセンターの音声ファイルから価値あるインサイトを抽出する強力な方法を紹介します。ここでは、以下のことを行います。

-   新しい`AI_TRANSCRIBE`関数を使用して音声ファイルを**文字起こし**する
-   Cortex Completeを使用して通話のパターンと傾向を**分析**する
-   構造化されたプロンプトを使用して主要な指標とインサイトを**抽出**する
-   目的、緊急度、満足度によって通話を**分類**する
-   異常や改善の機会を**発見**する
-   コールセンター最適化のための実用的な推奨事項を**生成**する

## AI_TRANSCRIBEの主な特徴
- 最新のAIモデルによる高品質な文字起こし
- 40以上の言語に対応
- 最大ファイルサイズ：700 MB
- 最大録音時間：90分
- .mp3 および .wav 形式をサポート

In [ ]:
# Import required packages
import pandas as pd
import json
import numpy as np
import time
from datetime import datetime, timedelta
import streamlit as st

from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.snowpark.version import VERSION

# Get active session
session = get_active_session()
session.use_role("accountadmin")
session.use_schema("analytics")

# Add query tag for monitoring
session.query_tag = {"origin":"sf_sit", "name":"call_center_analytics_2", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":1, "source":"notebook"}}

print(f"❄️ Snowflake Session Details:")
print(f"Role: {session.get_current_role()}")
print(f"Warehouse: {session.get_current_warehouse()}")
print(f"Database.Schema: {session.get_fully_qualified_current_schema()}")
print(f"Snowpark Version: {VERSION}")


## AI_TRANSCRIBEによる音声ファイルの文字起こし

ここでは、`AI_TRANSCRIBE`を使用して音声ファイルをテキストに変換します。FILEオブジェクトを格納するテーブルを作成し、バッチで処理します。

![img](https://lh3.googleusercontent.com/pw/AP1GczPInKocH8DMpk8nK8zCXKOxOU-g_hdO5sIDc8eCfipwTT_bf5VhIoPhcXbCKSyXvGDZvD2LTPYKRIkExCOiExqrChP3nuyj8DNztIcmEP18fJclD0IMFFEoNfr-s20RcAzblBSzYfdxeodB__qC66Y=w2676-h1508-s-no-gm?authuser=0)

In [ ]:
alter stage audio_files_ja refresh;
list @audio_files_ja;

In [ ]:
-- AI_TRANSCRIBEを実行するための FILE オブジェクトを含むテーブルを作成する
CREATE OR REPLACE TABLE audio_files_for_transcription_ja AS
SELECT 
    RELATIVE_PATH as file_path,
    TO_FILE('@audio_files_ja', RELATIVE_PATH) as audio_file,
    SIZE as file_size_bytes,
    LAST_MODIFIED as upload_time,
    SPLIT_PART(RELATIVE_PATH, '.', -1) as file_extension,
    REPLACE(RELATIVE_PATH, '.mp3', '') as call_id
FROM DIRECTORY('@audio_files_ja')
WHERE RELATIVE_PATH ILIKE '%.mp3' OR RELATIVE_PATH ILIKE '%.wav';


In [ ]:
-- AI 文字起こしを実行し、結果を保存する
-- 🎙️ AI 文字起こしプロセスを開始します...
CREATE OR REPLACE TABLE ai_transcribed_calls_ja AS
SELECT 
    call_id,
    file_path,
    file_size_bytes,
    upload_time,
    AI_TRANSCRIBE(audio_file) as transcription_result,
    transcription_result:text::STRING as transcript_text,
    CURRENT_TIMESTAMP() as transcription_timestamp,
    LENGTH(transcription_result:text::STRING) as transcript_length,
    CASE 
        WHEN transcription_result:text IS NULL THEN 'FAILED'
        WHEN LENGTH(transcription_result:text::STRING) < 10 THEN 'SHORT'
        ELSE 'SUCCESS'
    END as transcription_status
FROM audio_files_for_transcription_ja
ORDER BY file_size_bytes ASC;  

-- 大体40-60sで完了

In [ ]:
-- 内容を確認する
select * from ai_transcribed_calls_ja;

In [ ]:
# サンプルのオーディオ ファイルを再生してみましょう
# Snowflake ノートブックでは、Streamlit コンポーネントを直接使用できます
stage_path = "@call_center_analytics_db.analytics.audio_files_ja/CALL_20250728_10050_ja.mp3"

try:
    # 内部ステージからオーディオ ファイルを読み取ります
    with session.file.get_stream(stage_path) as f:
        audio_bytes = f.read()

    # st.audio を使用して MP3 を再生します
    # MP3 の形式を「audio/mpeg」として指定します
    st.audio(audio_bytes, format="audio/mpeg", start_time=0)

    st.success(f"Successfully loaded and playing: {stage_path}")

except Exception as e:
    st.error(f"Error loading or playing audio: {e}")
    st.info("Please ensure the audio file exists on the stage and the stage path is correct.")

In [ ]:
-- トランスクリプトの例を表示
SELECT
    transcript_text
FROM ai_transcribed_calls_ja
WHERE call_id = 'CALL_20250728_10050_ja';

In [ ]:
-- 文字起こし結果のサマリーを表示
SELECT 
    transcription_status,
    COUNT(*) as call_count,
    AVG(transcript_length) as avg_transcript_length
FROM ai_transcribed_calls_ja
GROUP BY transcription_status
ORDER BY call_count DESC;


## Cortex関数による高度な分析

### ここでは、様々なCortex AI関数を使用して、文字起こしされたデータから有意義なインサイトを抽出します。
### AI_SENTIMENT関数で感情分析を実施
![img](https://lh3.googleusercontent.com/pw/AP1GczNfs3g0EdxWd-rrxNUsc4qloRmIe6DUIYzSavgoi3-WNRucpaC6w3aMNxTiLUYc3FZxEJSMZuaHvmZKjzRE1VmkQYXJvEXeX42WexNevP1pWmNoaxE3N1PJTXCLqBm9eKjp5HxleU15NIlZyd1Ut1g=w2674-h1498-s-no-gm?authuser=0)

In [ ]:
WITH base AS (
    SELECT
        call_id,
        transcript_text,
        AI_SENTIMENT(
            transcript_text,
            ['Support Agent','Product','Company']
        ) AS sentiment_score
    FROM ai_transcribed_calls_ja
),
category_scores AS (
    SELECT
        call_id,
        sentiment_score,
        value:name::string  AS category_name,
        value:sentiment::string AS category_sentiment
    FROM base,
         LATERAL FLATTEN(input => sentiment_score:categories)
)
SELECT
    call_id,
    sentiment_score,
    MAX(CASE WHEN category_name = 'overall'       THEN category_sentiment END) AS overall_sentiment,
    MAX(CASE WHEN category_name = 'Company'       THEN category_sentiment END) AS company_sentiment,
    MAX(CASE WHEN category_name = 'Product'       THEN category_sentiment END) AS product_sentiment,
    MAX(CASE WHEN category_name = 'Support Agent' THEN category_sentiment END) AS support_agent_sentiment
FROM category_scores
GROUP BY call_id, sentiment_score;


### Complete関数を利用した複雑な処理
![img](https://lh3.googleusercontent.com/pw/AP1GczM5fvR3T9cm4akFBEgiVw4ShYsgQ1a6HzMip1zOjEccBORjQxKrVmgpg5iZ79BUk03EvQTIU8FhsBhX-Sar3rk2vzFVKakeFS7qUP1mW3Qm7A6Ut_jbgIh6Tt7rnYQimPM2WRAmG2gkBifPGNknsyA=w2674-h1498-s-no-gm?authuser=0)

In [ ]:
-- 🚀 文字起こしに対して最適化された高度な AI 分析を実行しています...
-- 単一のSQL ステートメントで最終テーブルを作成します。
CREATE OR REPLACE TABLE comprehensive_call_analysis_ja AS

-- Step 1: まず、AIに渡す選択肢となるエージェント名のリストをJSON配列文字列として作成
WITH agent_list_cte AS (
    SELECT
        TO_JSON(ARRAY_AGG(DISTINCT agent_name)) AS agent_list_json
    FROM
        call_center_analytics_db.analytics.support_agents
),

-- Step 2: AI分析の本体。CROSS JOINで全行にエージェントリストを結合し、動的にJSONスキーマを生成
base AS (
    SELECT
        call_id,
        transcript_text,
        -- word_count,
        AI_SENTIMENT(transcript_text, ['Support Agent','Product','Company']) AS sentiment_result,
        AI_COMPLETE(
            model => 'claude-4-sonnet', -- より高速でコスト効率の良いSonnetモデルを推奨
            prompt => '以下のコールセンターでの会話を分析し、指定されたJSON形式で包括的な情報を抽出してください。会話の書き起こし： ' || transcript_text,
            model_parameters => {'temperature': 0.1, 'max_tokens': 4096},
            -- ここで動的に生成したJSONスキーマ文字列をPARSE_JSONで渡す
            response_format => PARSE_JSON(
                -- 文字列結合(||)を使い、agent_nameのenum部分に動的にリストを埋め込む
                '{
                    "type": "json",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "call_summary": {"type": "string", "description": "この会話内容を100文字程度の箇条書きで要約してください。"},
                            "sentiment_score": {
                                "type": "number",
                                "description": "コールセンターの会話からセンチメントスコアを1〜10の10段階で総合評価してください。数値のスコアのみ。判断できない場合はnullを返す。"
                            },
                            "call_analysis": {
                                "type": "object",
                                "properties": {
                                    "call_type": {"type": "string", "enum": ["inbound", "outbound", "transfer"]},
                                    "customer_name": {"type": "string"},
                                    "agent_name": {"type": "string", "enum": ' || agent_list_cte.agent_list_json || '},
                                    "primary_intent": {"type": "string", "enum": ["billing", "technical_support", "complaint", "information", "sales", "cancellation", "other"]},
                                    "urgency_level": {"type": "string", "enum": ["low", "medium", "high", "critical"]},
                                    "issue_resolved": {"type": "string", "enum": ["yes", "no", "partial"]},
                                    "escalation_required": {"type": "string", "enum": ["yes", "no"]},
                                    "customer_satisfaction": {"type": "string", "enum": ["satisfied", "neutral", "dissatisfied"]},
                                    "call_duration_estimate": {"type": "string", "enum": ["short", "medium", "long"]},
                                    "key_issues": {"type": "array", "items": {"type": "string"}},
                                    "action_items": {"type": "array", "items": {"type": "string"}},
                                    "monetary_amounts": {"type": "array", "items": {"type": "string"}},
                                    "appointment_scheduled": {"type": "string", "enum": ["yes", "no"]},
                                    "callback_requested": {"type": "string", "enum": ["yes", "no"]}
                                }
                            },
                            "agent_performance_score": {
                                "type": "number",
                                "description": "オペレーターのパフォーマンスを1〜10の10段階で総合評価してください。数値のスコアのみ。判断できない場合はnullを返す。"
                            },
                            "improvement_opportunities": {
                                "type": "string",
                                "items": {"type": "string"},
                                "description": "この会話における、具体的な改善点を3つ箇条書きで挙げてください。"
                            }
                        },
                        "required": ["call_summary", "sentiment_score", "call_analysis", "agent_performance_score", "improvement_opportunities"]
                    }
                }'
            )
        ) AS ai_comprehensive_analysis,
        CURRENT_TIMESTAMP() AS analysis_timestamp
    FROM
        ai_transcribed_calls_ja
    -- CROSS JOINを使い、agent_list_cteの結果(1行)をai_transcribed_calls_jaの全行に結合する
    CROSS JOIN
        agent_list_cte
    WHERE
        transcription_status = 'SUCCESS'
        AND transcript_text IS NOT NULL
        AND LENGTH(transcript_text) > 50
),

-- Step 3: センチメントスコアを行から列に変換 (変更なし)
sentiments AS (
    SELECT
        b.call_id,
        MAX(CASE WHEN f.value:name::string = 'overall' THEN f.value:sentiment::string END) AS overall_sentiment,
        MAX(CASE WHEN f.value:name::string = 'Company' THEN f.value:sentiment::string END) AS company_sentiment,
        MAX(CASE WHEN f.value:name::string = 'Product' THEN f.value:sentiment::string END) AS product_sentiment,
        MAX(CASE WHEN f.value:name::string = 'Support Agent' THEN f.value:sentiment::string END) AS support_agent_sentiment
    FROM
        base AS b,
        LATERAL FLATTEN(input => b.sentiment_result:categories) AS f
    GROUP BY
        b.call_id
)

-- Step 4: 全てのデータを結合して最終的なテーブルを作成
SELECT
    b.call_id,
    b.transcript_text,
    b.sentiment_result,
    s.overall_sentiment,
    s.company_sentiment,
    s.product_sentiment,
    s.support_agent_sentiment,
    b.ai_comprehensive_analysis:call_summary::string AS call_summary,
    b.ai_comprehensive_analysis:sentiment_score::number AS sentiment_score,
    b.ai_comprehensive_analysis:call_analysis AS call_analysis,
    TRY_CAST(b.ai_comprehensive_analysis:agent_performance_score::varchar AS NUMBER(3,1)) AS agent_performance_score,
    b.ai_comprehensive_analysis:improvement_opportunities::variant AS improvement_opportunities,
    b.analysis_timestamp
FROM
    base b
LEFT JOIN
    sentiments s ON b.call_id = s.call_id;

In [ ]:
-- 内容を確認する
select * from comprehensive_call_analysis_ja;

In [ ]:
-- バリアントのカラムを展開する
ALTER TABLE comprehensive_call_analysis_ja
ADD COLUMN 
    call_type STRING,
    customer_name STRING,
    agent_name STRING,
    primary_intent STRING,
    urgency_level STRING,
    issue_resolved STRING,
    escalation_required STRING,
    customer_satisfaction STRING;

UPDATE comprehensive_call_analysis_ja
SET
    call_type = call_analysis:call_type::STRING,
    customer_name = call_analysis:customer_name::STRING,
    agent_name = call_analysis:agent_name::STRING,
    primary_intent = call_analysis:primary_intent::STRING,
    urgency_level = call_analysis:urgency_level::STRING,
    issue_resolved = call_analysis:issue_resolved::STRING,
    escalation_required = call_analysis:escalation_required::STRING,
    customer_satisfaction = call_analysis:customer_satisfaction::STRING;

In [ ]:
-- 再度確認する
select * from comprehensive_call_analysis_ja;

In [ ]:
-- 📊 分析のサマリー
SELECT 
    COUNT(*) as total_calls,
    ROUND(AVG(sentiment_score), 3) as avg_sentiment,
    ROUND(AVG(agent_performance_score), 1) as avg_agent_score,
    COUNT(DISTINCT agent_name) as unique_agents,
    COUNT(DISTINCT primary_intent) as unique_call_types
FROM comprehensive_call_analysis_ja;


## 重要な知見と傾向の発見

データを分析し、興味深い傾向や実用的な知見を明らかにしましょう。

In [ ]:
-- 🏆 エージェントごとのパフォーマンス分析
SELECT 
    agent_name,
    COUNT(*) as total_calls,
    ROUND(AVG(sentiment_score), 3) as avg_sentiment,
    ROUND(AVG(agent_performance_score), 1) as avg_performance_score,
    
    -- 解決率
    SUM(CASE WHEN issue_resolved = 'yes' THEN 1 ELSE 0 END) as resolved_calls,
    ROUND(SUM(CASE WHEN issue_resolved = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as resolution_rate,
    
    -- 顧客満足度
    SUM(CASE WHEN customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) as satisfied_customers,
    ROUND(SUM(CASE WHEN customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as satisfaction_rate,
    
    -- エスカレーション率
    SUM(CASE WHEN escalation_required = 'yes' THEN 1 ELSE 0 END) as escalations,
    ROUND(SUM(CASE WHEN escalation_required = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as escalation_rate
    
FROM comprehensive_call_analysis_ja
WHERE agent_name != 'Not Available' AND agent_name IS NOT NULL
GROUP BY agent_name
ORDER BY avg_performance_score DESC;


In [ ]:
-- 📊 コール内容別分析
WITH call_patterns AS (
    SELECT 
        primary_intent,
        urgency_level,
        COUNT(*) as call_count,
        ROUND(AVG(sentiment_score), 3) as avg_sentiment,
        ROUND(AVG(agent_performance_score), 1) as avg_agent_score,
        
        -- 解決率
        ROUND(SUM(CASE WHEN issue_resolved = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as resolution_rate,
        
        -- 満足率
        ROUND(SUM(CASE WHEN customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as satisfaction_rate,
        
        -- エスカレーション率
        ROUND(SUM(CASE WHEN escalation_required = 'yes' THEN 1 ELSE 0 END) / COUNT(*) * 100, 1) as escalation_rate
        
    FROM comprehensive_call_analysis_ja
    WHERE primary_intent IS NOT NULL AND primary_intent != 'Not Available'
    GROUP BY primary_intent, urgency_level
)
SELECT 
    primary_intent,
    urgency_level,
    call_count,
    avg_sentiment,
    avg_agent_score,
    resolution_rate || '%' as resolution_rate_pct,
    satisfaction_rate || '%' as satisfaction_rate_pct,
    escalation_rate || '%' as escalation_rate_pct,
    
    -- Performance flags
    CASE 
        WHEN resolution_rate < 70 THEN '⚠️ Low Resolution'
        WHEN satisfaction_rate < 60 THEN '⚠️ Low Satisfaction'
        WHEN escalation_rate > 30 THEN '⚠️ High Escalation'
        ELSE '✅ Good Performance'
    END as performance_flag
    
FROM call_patterns
ORDER BY call_count DESC;


## AIエージェントから情報にアクセスする

1. Cortex Search Serviceを作り、エージェントの応答内容にアクセス可能にする
2. セマンティックビューを作成し、エージェントの対応評価のデータにアクセス可能にする

## Create Search Serviceを作成する
### トランスクリプトをRAGとして検索できるようにする
![img](https://lh3.googleusercontent.com/pw/AP1GczPRuvMbEGh0M9ZQSASJKibWGnN1LNOjm8btACEMsfjowiXT9WGtCYYHSExza112XuavYa0O5MvCdtvVpfPb32V1lvH3dbMFeiOAGJ2z-LQjWJhdmdY9cjUeCQkiI5FhExd_lCZkVKWl80XC9y5u3ro=w2674-h1498-s-no-gm?authuser=0)

In [ ]:
-- 通話トランスクリプト用の Cortex 検索サービスを作成する
CREATE OR REPLACE CORTEX SEARCH SERVICE call_center_search_ja
ON transcript_text
ATTRIBUTES call_id, agent_name, customer_name, primary_intent, overall_sentiment, company_sentiment, product_sentiment, support_agent_sentiment, sentiment_score, customer_satisfaction
WAREHOUSE = call_center_analytics_wh
TARGET_LAG = '1 minute'
AS (
    SELECT 
        call_id,
        agent_name,
        customer_name, 
        primary_intent,
        overall_sentiment,
        company_sentiment,
        product_sentiment,
        support_agent_sentiment,
        sentiment_score,
        customer_satisfaction,
        transcript_text
    FROM comprehensive_call_analysis_ja
    WHERE transcript_text IS NOT NULL
    AND LENGTH(transcript_text) > 50
);

In [ ]:
-- サンプルクエリで検索サービスをテストする
SELECT
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
        'call_center_search_ja',
        '{
            "query": "請求に関する苦情に対する否定的な感情",
            "limit": 5
        }'
    ) as search_results;

## セマンティックビューを作成する
### センチメント情報やエスカレーション率などの定量データにアクセスする
![img](https://lh3.googleusercontent.com/pw/AP1GczPLDCXT0UdRP20P8av1qvA2Ld40EDCshCFGIFKGeH82LCizXLFCfmR_onmUbAzR0a0lI-qt4v-LTyZ6eWwZU40gz-ejdf5BjVGxMgyZVNk8GUwkgAUdb6Ft4-065vtY3nIfg_iihECuH6X6YoBMd8g=w2674-h1498-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW CALL_CENTER_ANALYTICS_DB.ANALYTICS.CALL_CENTER_ANALYTICS_MODEL_JA
  TABLES (
    cca AS CALL_CENTER_ANALYTICS_DB.ANALYTICS.COMPREHENSIVE_CALL_ANALYSIS_JA
      PRIMARY KEY (CALL_ID)
      COMMENT = 'Main table containing transcribed call data with AI-extracted insights including sentiment analysis, agent performance metrics, and customer satisfaction scores'
  )

  FACTS (
    cca.sentiment_score AS SENTIMENT_SCORE
      COMMENT = 'Numerical sentiment score for the call on a 1–10 scale',
    cca.agent_performance_score AS AGENT_PERFORMANCE_SCORE
      COMMENT = 'Performance score for the agent on a 1–10 scale'
  )

  DIMENSIONS (
    cca.call_id AS CALL_ID
      COMMENT = 'Unique identifier for each call',
    cca.agent_name AS AGENT_NAME
      COMMENT = 'Name of the customer service agent handling the call',
    cca.customer_name AS CUSTOMER_NAME
      COMMENT = 'Name of the customer making the call',
    cca.call_type AS CALL_TYPE
      COMMENT = 'Inbound / outbound / transfer classification',
    cca.primary_intent AS PRIMARY_INTENT
      COMMENT = 'Primary reason for the call (billing, technical_support, complaint, information, sales, cancellation, other)',
    cca.urgency_level AS URGENCY_LEVEL
      COMMENT = 'Urgency level of the call',
    cca.issue_resolved AS ISSUE_RESOLVED
      COMMENT = 'Whether the customer issue was resolved during the call',
    cca.escalation_required AS ESCALATION_REQUIRED
      COMMENT = 'Whether the call required escalation to higher level support',
    cca.customer_satisfaction AS CUSTOMER_SATISFACTION
      COMMENT = 'Customer satisfaction level determined from call analysis',
    cca.overall_sentiment AS OVERALL_SENTIMENT
      COMMENT = 'Overall sentiment classification of the conversation',
    cca.company_sentiment AS COMPANY_SENTIMENT
      COMMENT = 'Sentiment toward the company',
    cca.product_sentiment AS PRODUCT_SENTIMENT
      COMMENT = 'Sentiment toward the product',
    cca.support_agent_sentiment AS SUPPORT_AGENT_SENTIMENT
      COMMENT = 'Sentiment toward the support agent',
    cca.analysis_timestamp AS ANALYSIS_TIMESTAMP
      COMMENT = 'Timestamp when the call analysis was completed'
  )

  METRICS (
    cca.total_calls AS COUNT(*)
      COMMENT = 'Total number of calls',
    cca.avg_sentiment_score AS AVG(cca.sentiment_score)
      COMMENT = 'Average sentiment score across calls',
    cca.avg_agent_performance AS AVG(cca.agent_performance_score)
      COMMENT = 'Average agent performance score',
    cca.resolution_rate AS 100 * SUM(CASE WHEN cca.issue_resolved = 'yes' THEN 1 ELSE 0 END) / NULLIF(COUNT(*),0)
      COMMENT = 'Percentage of calls with issues resolved successfully',
    cca.satisfaction_rate AS 100 * SUM(CASE WHEN cca.customer_satisfaction = 'satisfied' THEN 1 ELSE 0 END) / NULLIF(COUNT(*),0)
      COMMENT = 'Percentage of customers who were satisfied with the call',
    cca.escalation_rate AS 100 * SUM(CASE WHEN cca.escalation_required = 'yes' THEN 1 ELSE 0 END) / NULLIF(COUNT(*),0)
      COMMENT = 'Percentage of calls that required escalation',
    cca.positive_sentiment_rate AS 100 * SUM(CASE WHEN LOWER(cca.overall_sentiment) = 'positive' THEN 1 ELSE 0 END) / NULLIF(COUNT(*),0)
      COMMENT = 'Percentage of calls with positive overall sentiment'
  )

  COMMENT = 'Semantic model for call center analytics with AI transcription data, enabling natural language queries about call performance, agent effectiveness, and customer satisfaction'
  COPY GRANTS;

## Cortex Agentを作成する
### 上記で作成したCortex SearchとCortex AnalystをツールとしたAIエージェントを作成する
![img](https://lh3.googleusercontent.com/pw/AP1GczMtNlFme2xdV6s8oBUYkWB9rXr-Q906Y0HQTdVB2SUe_zct6C7sUQZlsofw_V2iHVicSWWi4KAWVWdlFOJ1CdkmokQ3Q7GFmqIDe-gwJ_APvqtRDUjLXfDzgMLwENyJ-UUswd0xj5oxXTaAoLHDAJg=w2674-h1498-s-no-gm?authuser=0)

In [ ]:
CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.CALL_CENTER_AGENT_JA
  COMMENT = 'コールセンター スーパーバイザー エージェント 　エージェントの対応評価のアクセスにはSemantic View を使用し、対応中のトランスクリプトのアクセスには Cortex Search を使用します。'
  PROFILE = '{"display_name": "コールセンター監督エージェント", "avatar": "headset.png", "color": "blue"}'
  FROM SPECIFICATION
$$
# モデル指定は省略（自動選択）

orchestration:
  budget:
    seconds: 45
    tokens: 24000

instructions:
  system: |
    あなたはコールセンターのスーパーバイザーとして行動する。
    目的はサポート担当者の業務改善である。担当者・顧客・通話の評価を正しく把握し、
    改善提案、傾向の把握、エスカレーション抑制、顧客満足度の向上につながる助言を行う。
    数値や割合の質問はセマンティックビューを使ってSQLで集計する。
    会話内容の検証や根拠提示が必要なときはCortex Searchでトランスクリプトを参照する。
    回答では数値に単位と期間を明記し、根拠テキストがある場合は短い抜粋を示す。
  orchestration: |
    「件数・平均・率・推移・〜別」はAnalystを優先。
    「この通話の内容・発言の根拠・抜粋提示」はSearchを優先。
    改善提案を求められた場合は、まず現状をAnalystで把握し、必要に応じてSearchで具体例を補足する。
  response: |
    日本語で簡潔に答える。専門用語を避ける。箇条書きは求められた場合のみ。

tools:
  - tool_spec:
      type: "cortex_analyst_text_to_sql"
      name: "AnalystJA"
      description: "自然言語→SQLでセマンティックビューを集計"
  - tool_spec:
      type: "cortex_search"
      name: "SearchJA"
      description: "通話トランスクリプト全文検索と要約"

tool_resources:
  AnalystJA:
    semantic_view: "CALL_CENTER_ANALYTICS_DB.ANALYTICS.CALL_CENTER_ANALYTICS_MODEL_JA"
  SearchJA:
    name: "CALL_CENTER_ANALYTICS_DB.ANALYTICS.call_center_search_ja"
    max_results: "8"
    title_column: "primary_intent"
    id_column: "call_id"
$$;


## まとめと次のステップ

### このノートブックでは、まずAI_TRANSCRIBEとSnowflakeのCortex AI関数を組み合わせることで、以下の強力な機能が実現できることを示しました。

✅ コールセンターの音声ファイルを高精度で**自動的に文字起こし**する  
✅ 非構造化の会話データから**構造化された知見を抽出**する  
✅ **パフォーマンスの傾向**と改善機会を特定する  
✅ 異常や品質問題を**自動的に検知**する  
✅ AI分析を用いて**実用的な改善提案を生成**する  
✅ 経営層の意思決定のために**包括的なレポートを作成**する  

---

### AI_TRANSCRIBEの主な利点：
- **インフラ管理が不要** - サーバーレスな文字起こし
- **多言語対応** - 40以上の言語をサポート
- **高精度** - 最新のAIモデルによる文字起こし
- **スケーラブルな処理** - 大量の音声ファイルに対応
- **統合された分析** - 他のCortex関数とのシームレスな連携


### 次に、Cortex Search Serviceを作り、応答内容を検索可能に、セマンティックビューを作成し、構造化に整理したセンチメント情報などをアクセス可能にしました
### Cortex Agentに上記のCortex Search Serviceとセマンティックビューを紐づけ、AIエージェントから利用できるようにしています

---

### 🔍 **Cortex Searchサービス**
- **名前**： `call_center_search`
- **目的**： 通話記録に対するセマンティック検索
- **用途**： 自然言語クエリに基づき、関連する通話を発見

---

### 📊 **セマンティックビュー**
- **場所**： `CALL_CENTER_ANALYTICS_DB.ANALYTICS.CALL_CENTER_ANALYTICS_MODEL_JA`
- **目的**： 自然言語からSQLへのクエリ変換を有効化
- **用途**： コールセンターの指標について平易な言葉で質問

---

### 📈 **作成されたビュー**
- `call_center_kpis`： 全体的なKPIの概要
- `agent_performance_summary`： 詳細分析のためのエージェントのパフォーマンス指標


## 次のステップ
### 作成したCortex Agentの詳細を確認し、Snowflake Intelligenceから利用可能にします
左側のメニューから、AIとML > エージェントを選択し、CALL_CENTER_AGENT_JAのエージェントを選択して詳細を確認します